In [3]:
import time
import requests
import os
import datetime
import yaml

from rich import print
from dotenv import load_dotenv

import dspy
from dspy import (
    Predict,
    Signature,
    ChainOfThought,
    LM,
    settings
)

load_dotenv('./.env')
nai_api_key = os.getenv('NAI_API_KEY')
# print(nai_api_key)
hf_api_key = os.getenv('HF_TOKEN')

nai_url= 'https://ai.nutanix.com/api/v1/chat/completions'

/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
source_code_file = '/Users/hritik.raj/ai-hpc-codegen-unittest/automated_unittesting_app/codegen-observation/hritik/health.go/health.go'
source_code_unit_test_file ='/Users/hritik.raj/ai-hpc-codegen-unittest/automated_unittesting_app/codegen-observation/hritik/health.go/health_test_codegen.go'

In [18]:
with open('Prompts/prompt_01.yaml', 'r') as file:
    prompts = yaml.safe_load(file)

with open(source_code_file, 'r') as source_file:
    go_source_code = source_file.read()

with open(source_code_unit_test_file, 'r') as test_file:
    go_unit_test_code = test_file.read()

print(prompts)
system_prompt = prompts.get('system_prommpt','')

user_prompt = prompts.get('user_prompt', '')
user_prompt = user_prompt.replace('<GO_SOURCE_CODE>', go_source_code).replace('<GO_UNIT_TEST_CODE>', go_unit_test_code)

print(user_prompt)

{
    'system_prompt': '### You are an expert in analyzing the unit tests for a given Go code and its corresponding 
unit test cases. ###\nProvide detailed insights, potential issues, and recommendations to improve coverage and 
correctness.\n',
    'user_prompt': '---------------------------\n\nHere is the Go source 
code:\n\n---------------------------\n\n<GO_SOURCE_CODE>\n\n----------------------------\n\nHere is the Go unit 
test code:\n\n---------------------------\n\n<GO_UNIT_TEST_CODE>\n'
}

---------------------------

Here is the Go source code:

---------------------------

package client

import (
        "context"
        "net/http"
        "time"

        "github.com/nutanix-core/nai-api/iep/constants"
        "github.com/nutanix-core/nai-api/iep/constants/enum"
)

// IHealthClient interface contains methods to fetch inference endpoint health
type IHealthClient interface {
        CheckHealth(healthCheckURL string) enum.ServiceHealthStatusCode
}

// healthClient struct client executes health api calls on inference endpoints
type healthClient struct {
        client IClient
}

// NewHealthClient instantiates Inference client
func NewHealthClient(client IClient) IHealthClient {
        return &healthClient{
                client: client,
        }
}

// CheckHealth executes health api call for a given inference endpoint
func (ic *healthClient) CheckHealth(healthCheckURL string) enum.ServiceHealthStatusCode {
        ic.client.SetTimeout(constants.ClientTimeout * time.Second)

        var status enum.ServiceHealthStatusCode

        // if service is unreachable/unhealthy, retry for MaxServiceHealthRetries times
        for retry := 1; retry <= constants.MaxServiceHealthAttempts; retry++ {
                status = ic.checkHealthInternal(healthCheckURL)

                if status == enum.HealthyStatusCode {
                        return status
                }
                // retry if error occurred while checking health or service is unhealthy
                // sleep before retrying, do not sleep after last retry
                if retry < constants.MaxServiceHealthAttempts {
                        time.Sleep(constants.ServiceHealthCheckRetryDelay * time.Second)
                }
        }
        return status
}

func (ic *healthClient) checkHealthInternal(healthCheckURL string) enum.ServiceHealthStatusCode {
        req, err := http.NewRequest(http.MethodGet, healthCheckURL, nil)
        if err != nil {
                // returning status as unknown as http request creation failed, hence the status is unknown
                return enum.UnknownStatusCode
        }

        resp, _, err := ic.client.Do(context.Background(), req)

        // endpoint health is critical as health api failed
        if err != nil {
                return enum.CriticalStatusCode
        }

        // no error while executing request
        defer resp.Body.Close() //nolint:errcheck

        // check response status code
        if resp.StatusCode != http.StatusOK {
                return enum.CriticalStatusCode
        }

        return enum.HealthyStatusCode
}


----------------------------

Here is the Go unit test code:

---------------------------

package client

import (
        "context"
        "net/http"
        "testing"
)

func TestNewHealthClient(t *testing.T) {
        client := &IClient{}
        ic := NewHealthClient(client)
        _, ok := ic.(*healthClient)
        require.True(t, ok)
}

func TestCheckHealth(t *testing.T) {
        client := &IClient{}
        ic := NewHealthClient(client)
        req, err := http.NewRequest(http.MethodGet, "http://example.com", nil)
        require.NoError(t, err)
        resp, _, err := ic.client.Do(context.Background(), req)
        require.NoError(t, err)
        defer resp.Body.Close()
        require.Equal(t, http.StatusOK, resp.StatusCode)
}

func TestCheckHealthInternal(t *testing.T) {
        client := &IClient{}
        ic := NewHealthClient(client)
        req, err := http.NewRequest(http.MethodGet, "http://example.com", nil)
        require.NoError(t, err)
        resp, _, err := ic.client.Do(context.Background(), req)
        require.NoError(t, err)
        defer resp.Body.Close()
        require.Equal(t, http.StatusOK, resp.StatusCode)
}

func TestCheckHealthInternalError(t *testing.T) {
        client := &IClient{}
        ic := NewHealthClient(client)
        req, err := http.NewRequest(http.MethodGet, "http://example.com", nil)
        require.NoError(t, e

In [19]:


inference_input = {
    "model": "vllm-llama-3-1",
    "messages": [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    "max_tokens": 256,
    "stream": False
}


response = requests.post(nai_url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json=inference_input, 
                         verify=False)

/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
response_from_model = response.json()["choices"][0]["message"]["content"]
print(response_from_model)

There are a few issues with the unit test code:

1.  **Missing setup and execution of the health check**:

    In `TestCheckHealth`, `TestCheckHealthInternal`, `TestCheckHealthInternalError`, and 
`TestCheckHealthInternalUnknownStatusCode`, the `checkHealthInternal` method of `ic` is not called, but instead, 
the code looks like it's directly calling `ic.client.Do`. The correct test should call the `CheckHealth` method to 
test if the health check is being performed correctly.

2.  **Not checking the status code return value**:

    In the `TestCheckHealth`, `TestCheckHealthInternal`, `TestCheckHealthInternalError`, and 
`TestCheckHealthInternalUnknownStatusCode` tests, the `checkHealthInternal` method is directly called (with a 
slight modification) and the status code is verified. However, in the `CheckHealth` method, the status code is 
obtained after a loop with retries. The tests should be modified to call the `CheckHealth` method and verify the 
returned status code.

3.  **No test for edge case i.e. client timeout**:

    The `CheckHealth` method sets a timeout for the API call using `ic.client.SetTimeout`. To ensure that

In [39]:
with open('prompts/prompt_01.prompt', 'r') as file:
    base_prompt = file.read()

print(base_prompt)

Analyze the following sales report and provide a summary of the top 3 performing products along with their respective revenue
